# 🎤 Codename RVC Fork 4 - Modo CLI (Google Colab)

##### Notebook made by shiroug :shiba:

> 📁 **Esta versão tem suporte ao Google Drive** para salvar seus modelos e datasets!

In [ ]:
# @title 1️⃣ Montar Google Drive e Clonar Repositório
# @markdown Monta o Drive, clona o repositório e cria symlinks para persistência

MOUNT_DRIVE = True  # @param {type:"boolean"}
DRIVE_FOLDER = "codename-rvc"  # @param {type:"string"}

import os
import shutil
from pathlib import Path

%cd /content

DRIVE_PATH = None

# === MONTAR GOOGLE DRIVE ===
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    DRIVE_PATH = Path(f"/content/drive/MyDrive/{DRIVE_FOLDER}")
    DRIVE_PATH.mkdir(parents=True, exist_ok=True)

    # Criar subpastas no Drive
    (DRIVE_PATH / "datasets").mkdir(exist_ok=True)
    (DRIVE_PATH / "logs").mkdir(exist_ok=True)  # Modelos treinados
    (DRIVE_PATH / "pretrains").mkdir(exist_ok=True)
    (DRIVE_PATH / "outputs").mkdir(exist_ok=True)

    print(f"✅ Google Drive montado em: {DRIVE_PATH}")
else:
    print("⚠️ Drive não montado. Arquivos serão perdidos ao desconectar.")

# === CLONAR REPOSITÓRIO ===
print("\n📥 Clonando repositório...")
!git clone https://github.com/ShiromiyaG/codename-rvc-fork-4.git
%cd codename-rvc-fork-4

# === CRIAR SYMLINKS PARA O DRIVE ===
if MOUNT_DRIVE and DRIVE_PATH:
    print("\n🔗 Criando symlinks para o Drive...")

    # Symlink logs/ -> Drive/logs/ (modelos treinados)
    logs_local = Path("logs")
    logs_drive = DRIVE_PATH / "logs"
    if logs_local.exists() or logs_local.is_symlink():
        if logs_local.is_symlink():
            logs_local.unlink()
        else:
            shutil.rmtree(logs_local)
    os.symlink(logs_drive, logs_local)
    print(f"   ✅ logs/ -> {logs_drive}")

    # Symlink assets/datasets/ -> Drive/datasets/
    datasets_local = Path("assets/datasets")
    datasets_drive = DRIVE_PATH / "datasets"
    datasets_local.parent.mkdir(parents=True, exist_ok=True)
    if datasets_local.exists() or datasets_local.is_symlink():
        if datasets_local.is_symlink():
            datasets_local.unlink()
        else:
            shutil.rmtree(datasets_local)
    os.symlink(datasets_drive, datasets_local)
    print(f"   ✅ assets/datasets/ -> {datasets_drive}")

    # Symlink rvc/models/pretraineds/custom/ -> Drive/pretrains/
    pretrains_local = Path("rvc/models/pretraineds/custom")
    pretrains_drive = DRIVE_PATH / "pretrains"
    pretrains_local.parent.mkdir(parents=True, exist_ok=True)
    if pretrains_local.exists() or pretrains_local.is_symlink():
        if pretrains_local.is_symlink():
            pretrains_local.unlink()
        else:
            shutil.rmtree(pretrains_local)
    os.symlink(pretrains_drive, pretrains_local)
    print(f"   ✅ rvc/models/pretraineds/custom/ -> {pretrains_drive}")

    print("\n📁 Estrutura no Drive:")
    print(f"   {DRIVE_PATH}/")
    print(f"   ├── datasets/   <- Coloque seus áudios aqui")
    print(f"   ├── logs/       <- Modelos treinados (automático)")
    print(f"   ├── pretrains/  <- Pretrains G/D custom")
    print(f"   └── outputs/    <- Áudios convertidos")

# === INSTALAR DEPENDÊNCIAS ===
print("\n🔄 Configurando Python 3.10...")
!apt-get update -qq
!apt-get install -y -qq python3.10 python3.10-dev python3.10-distutils libpython3.10-dev curl aria2
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!update-alternatives --set python3 /usr/bin/python3.10

print("\n📦 Instalando dependências...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ["PATH"] += ":/root/.cargo/bin"
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
!uv pip install --system -r requirements.txt
!uv pip install ring-attention-pytorch

print("\n✅ Configuração concluída!")
!python3 --version

In [ ]:
# @title 2️⃣ Baixar Pretrains Customizados (Hugging Face)

# @markdown ### Gerador (G)
G_url = "" # @param {type:"string"}
G_filename = "G_custom.pth" # @param {type:"string"}

# @markdown ### Discriminador (D)
D_url = "" # @param {type:"string"}
D_filename = "D_custom.pth" # @param {type:"string"}

# @markdown ### Salvar no Google Drive?
SAVE_TO_DRIVE = True # @param {type:"boolean"}

import subprocess
from pathlib import Path

# Escolher pasta de destino
if SAVE_TO_DRIVE and DRIVE_PATH:
    PRETRAIN_DIR = DRIVE_PATH / "pretrains"
else:
    PRETRAIN_DIR = Path("rvc/models/pretraineds/custom")

def download_file(url, output_dir, filename):
    """Baixa um arquivo focado em links do Hugging Face"""
    if not url.strip():
        return

    if "huggingface.co" in url:
        url = url.replace("/blob/", "/resolve/")
        if "?" in url:
            url = url.split("?")[0]

    output_dir.mkdir(parents=True, exist_ok=True)
    full_path = output_dir / filename

    if full_path.exists():
        print(f"⏭️ {filename} já existe")
        return

    print(f"📥 Baixando {filename}...")
    !wget -q -O "{full_path}" "{url}"

    if full_path.exists():
        print(f"✅ {filename} baixado em: {full_path}")

if G_url:
    download_file(G_url, PRETRAIN_DIR, G_filename)
if D_url:
    download_file(D_url, PRETRAIN_DIR, D_filename)

# Criar symlink para o projeto se salvou no Drive
if SAVE_TO_DRIVE and DRIVE_PATH:
    local_pretrain = Path("rvc/models/pretraineds/custom")
    local_pretrain.mkdir(parents=True, exist_ok=True)
    for f in PRETRAIN_DIR.glob("*.pth"):
        dest = local_pretrain / f.name
        if not dest.exists():
            !cp "{f}" "{dest}"
            print(f"📋 Copiado {f.name} para projeto")

---
# 🎓 FLUXO DE TREINO

O pipeline de treino consiste em 3 etapas:
1. **Preprocess** → Preparar e limpar arquivos de áudio
2. **Extract** → Extrair pitch (F0) e embeddings de voz  
3. **Train** → Treinar o modelo

---

In [ ]:
# @title 3️⃣ Etapa 1: Preprocessar Dataset
# @markdown Prepara os arquivos de áudio para treino
# @markdown
# @markdown > ⚠️ **Nota:** Se `CUT_MODE = "Simple"`, os áudios terão o silêncio truncado automaticamente.

# @markdown ### Configurações Obrigatórias
MODEL_NAME = "meu_modelo"  # @param {type:"string"}

# @markdown ### Origem do Dataset
USE_DRIVE_DATASET = True  # @param {type:"boolean"}
DATASET_NAME = "meu_dataset"  # @param {type:"string"}

SAMPLE_RATE = 32000  # @param [24000, 32000, 40000, 48000] {type:"raw"}

# @markdown ### Configurações Opcionais
CUT_MODE = "Simple"  # @param ["Skip", "Simple", "Automatic"]
NORMALIZATION = "post_lufs"  # @param ["none", "post_lufs", "post_peak"]
CHUNK_LEN = 3.0  # @param {type:"slider", min:0.5, max:5.0, step:0.5}

# @markdown ### Truncar Silêncio (apenas para Simple)
SILENCE_THRESHOLD_DB = -50  # @param {type:"slider", min:-70, max:-20, step:5}
MIN_SILENCE_DURATION = 0.5  # @param {type:"slider", min:0.1, max:2.0, step:0.1}
KEEP_SILENCE = 0.3  # @param {type:"slider", min:0.0, max:1.0, step:0.1}

import os
import shutil
from pathlib import Path
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import warnings
warnings.filterwarnings('ignore')

# Determinar caminho do dataset
if USE_DRIVE_DATASET and DRIVE_PATH:
    DATASET_PATH = str(DRIVE_PATH / "datasets" / DATASET_NAME)
else:
    DATASET_PATH = f"assets/datasets/{DATASET_NAME}"

def truncate_silence(audio_path, output_path, threshold_db=-50, min_silence_ms=500, keep_silence_ms=300):
    try:
        audio = AudioSegment.from_file(audio_path)
        nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_ms, silence_thresh=threshold_db)

        if not nonsilent_ranges:
            return False

        start_ms = max(0, nonsilent_ranges[0][0] - keep_silence_ms)
        end_ms = min(len(audio), nonsilent_ranges[-1][1] + keep_silence_ms)

        trimmed_audio = audio[start_ms:end_ms]
        output_path.parent.mkdir(parents=True, exist_ok=True)
        trimmed_audio.export(output_path, format=output_path.suffix[1:])
        return True
    except Exception as e:
        print(f"   ⚠️ Erro: {audio_path.name}: {e}")
        return False

def process_dataset_silence(dataset_path, threshold_db, min_silence_sec, keep_silence_sec):
    dataset_path = Path(dataset_path)
    temp_dataset = Path(f"/tmp/dataset_trimmed_{dataset_path.name}")

    if temp_dataset.exists():
        shutil.rmtree(temp_dataset)
    temp_dataset.mkdir(parents=True, exist_ok=True)

    audio_extensions = {'.wav', '.mp3', '.flac', '.ogg', '.m4a', '.opus'}
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(dataset_path.rglob(f"*{ext}"))

    if not audio_files:
        print("⚠️ Nenhum arquivo de áudio encontrado!")
        return None

    print(f"🔇 Truncando silêncio de {len(audio_files)} arquivos...")

    min_silence_ms = int(min_silence_sec * 1000)
    keep_silence_ms = int(keep_silence_sec * 1000)

    processed = 0
    for audio_file in audio_files:
        relative_path = audio_file.relative_to(dataset_path)
        output_path = temp_dataset / relative_path
        if truncate_silence(audio_file, output_path, threshold_db, min_silence_ms, keep_silence_ms):
            processed += 1

    print(f"✅ {processed} arquivos processados")
    return str(temp_dataset)

print(f"📂 Modelo: {MODEL_NAME}")
print(f"📂 Dataset: {DATASET_PATH}")
print(f"🎵 Sample Rate: {SAMPLE_RATE}Hz")
print("="*60)

final_dataset_path = DATASET_PATH

if CUT_MODE == "Simple":
    print("\n🔇 Truncando silêncio...")
    processed_path = process_dataset_silence(
        DATASET_PATH,
        threshold_db=SILENCE_THRESHOLD_DB,
        min_silence_sec=MIN_SILENCE_DURATION,
        keep_silence_sec=KEEP_SILENCE
    )
    if processed_path:
        final_dataset_path = processed_path

print("\n🚀 Iniciando preprocessamento...\n")

!python core.py preprocess \
    --model_name "{MODEL_NAME}" \
    --dataset_path "{final_dataset_path}" \
    --sample_rate {SAMPLE_RATE} \
    --cut_preprocess "{CUT_MODE}" \
    --normalization_mode "{NORMALIZATION}" \
    --chunk_len {CHUNK_LEN} \
    --loading_resampling "ffmpeg" \
    --lufs_range_finder True

print("\n✅ Preprocessamento concluído!")

In [ ]:
# @title 4️⃣ Etapa 2: Extrair Features
# @markdown Extrai pitch (F0) e embeddings de voz

F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
EMBEDDER = "contentvec"  # @param ["contentvec", "spin_v1", "spin_v2", "custom"]
VOCODER_ARCH = "hifi_refine"  # @param ["hifi_refine", "ringformer", "pcph_gan"]

print(f"📂 Modelo: {MODEL_NAME}")
print(f"🎤 F0: {F0_METHOD} | Embedder: {EMBEDDER}")
print("="*60)

!python core.py extract \
    --model_name "{MODEL_NAME}" \
    --sample_rate {SAMPLE_RATE} \
    --f0_method "{F0_METHOD}" \
    --embedder_model "{EMBEDDER}" \
    --vocoder_arch "{VOCODER_ARCH}" \
    --include_mutes 2 \
    --gpu "0"

print("\n✅ Extração concluída!")

In [ ]:
# @title 5️⃣ Etapa 3: Treinar Modelo

# @markdown ### Configurações
TOTAL_EPOCHS = 300  # @param {type:"integer"}
BATCH_SIZE = 8  # @param {type:"slider", min:1, max:32, step:1}

# @markdown ### Arquitetura
VOCODER = "HiFi-GAN"  # @param ["HiFi-GAN", "PCPH-GAN", "RefineGAN", "RingFormer_v1", "RingFormer_v2"]
ARCHITECTURE = "RVC"  # @param ["RVC", "Fork/Applio", "Fork"]
OPTIMIZER = "AdamW"  # @param ["AdamW BF16", "AdamW", "AdamSPD", "RAdam", "Ranger21", "DiffGrad", "Prodigy"]

# @markdown ### Modelo Pré-treinado
USE_PRETRAINED = True  # @param {type:"boolean"}
CUSTOM_PRETRAINED = False  # @param {type:"boolean"}
G_PATH = "rvc/models/pretraineds/custom/G_custom.pth"  # @param {type:"string"}
D_PATH = "rvc/models/pretraineds/custom/D_custom.pth"  # @param {type:"string"}

# @markdown ### Salvamento
SAVE_EVERY = 10  # @param {type:"integer"}
SAVE_ONLY_LATEST = True  # @param {type:"boolean"}

# @markdown ### Avançado
USE_WARMUP = False  # @param {type:"boolean"}
WARMUP_EPOCHS = 10  # @param {type:"integer"}
SPECTRAL_LOSS = "L1 Mel Loss"  # @param ["L1 Mel Loss", "Multi-Scale Mel Loss", "Multi-Res STFT Loss"]
LR_SCHEDULER = "exp decay step"  # @param ["exp decay step", "exp decay epoch", "cosine annealing", "none"]
VITS_2_MODE = False # @param {type:"boolean"}
CLEANUP = True # @param {type:"boolean"}

print(f"📂 Modelo: {MODEL_NAME}")
print(f"🔢 Epochs: {TOTAL_EPOCHS} | Batch: {BATCH_SIZE}")
print("="*60)

cmd = f"""python core.py train \\
    --model_name "{MODEL_NAME}" \\
    --sample_rate {SAMPLE_RATE} \\
    --total_epoch_count {TOTAL_EPOCHS} \\
    --batch_size {BATCH_SIZE} \\
    --vocoder "{VOCODER}" \\
    --architecture "{ARCHITECTURE}" \\
    --optimizer "{OPTIMIZER}" \\
    --epoch_save_frequency {SAVE_EVERY} \\
    --save_only_latest_net_models {str(SAVE_ONLY_LATEST)} \\
    --save_weight_models True \\
    --pretrained {str(USE_PRETRAINED)} \\
    --custom_pretrained {str(CUSTOM_PRETRAINED)} \\
    --use_warmup {str(USE_WARMUP)} \\
    --warmup_duration {WARMUP_EPOCHS} \\
    --spectral_loss "{SPECTRAL_LOSS}" \\
    --lr_scheduler "{LR_SCHEDULER}" \\
    --use_validation False \\
    --vits_2_mode {str(VITS_2_MODE)} \\
    --cleanup {str(CLEANUP)} \\
    --gpu "0"""

if CUSTOM_PRETRAINED and G_PATH:
    cmd += f' --g_pretrained_path "{G_PATH}"'
if CUSTOM_PRETRAINED and D_PATH:
    cmd += f' --d_pretrained_path "{D_PATH}"'

print("\n🚀 Iniciando treino...")
!{cmd}

In [ ]:
# @title 6️⃣ Gerar Index e Salvar no Drive

INDEX_ALGORITHM = "Auto"  # @param ["Auto", "Faiss", "KMeans"]
SAVE_MODEL_TO_DRIVE = True  # @param {type:"boolean"}

print(f"📂 Gerando index para: {MODEL_NAME}")
print("="*60)

!python core.py index \
    --model_name "{MODEL_NAME}" \
    --index_algorithm "{INDEX_ALGORITHM}"

print("\n✅ Index gerado!")

# Copiar modelo para o Drive
if SAVE_MODEL_TO_DRIVE and DRIVE_PATH:
    import shutil
    from pathlib import Path

    model_log_dir = Path(f"logs/{MODEL_NAME}")
    drive_model_dir = DRIVE_PATH / "models" / MODEL_NAME

    if model_log_dir.exists():
        print(f"\n💾 Copiando modelo para Google Drive...")

        drive_model_dir.mkdir(parents=True, exist_ok=True)

        # Copiar arquivos .pth e .index
        for f in model_log_dir.glob("*.pth"):
            shutil.copy2(f, drive_model_dir / f.name)
            print(f"   ✅ {f.name}")

        for f in model_log_dir.glob("*.index"):
            shutil.copy2(f, drive_model_dir / f.name)
            print(f"   ✅ {f.name}")

        print(f"\n📂 Modelo salvo em: {drive_model_dir}")
    else:
        print(f"⚠️ Pasta do modelo não encontrada: {model_log_dir}")

---
# 🎙️ INFERÊNCIA (Conversão de Voz)

Converta áudios usando seu modelo treinado.

---

In [ ]:
# @title 7️⃣ Carregar Modelo do Drive
# @markdown Use esta célula para carregar um modelo salvo anteriormente

LOAD_MODEL_NAME = "meu_modelo"  # @param {type:"string"}

if DRIVE_PATH:
    import shutil
    from pathlib import Path

    drive_model_dir = DRIVE_PATH / "models" / LOAD_MODEL_NAME
    local_model_dir = Path(f"logs/{LOAD_MODEL_NAME}")

    if drive_model_dir.exists():
        print(f"📂 Carregando modelo do Drive: {LOAD_MODEL_NAME}")

        local_model_dir.mkdir(parents=True, exist_ok=True)

        for f in drive_model_dir.glob("*"):
            if f.suffix in ['.pth', '.index']:
                shutil.copy2(f, local_model_dir / f.name)
                print(f"   ✅ {f.name}")

        print(f"\n✅ Modelo carregado em: {local_model_dir}")

        # Atualizar variáveis
        MODEL_NAME = LOAD_MODEL_NAME
    else:
        print(f"⚠️ Modelo não encontrado no Drive: {drive_model_dir}")
else:
    print("⚠️ Google Drive não montado!")

In [ ]:
# @title 8️⃣ Inferência de Arquivo Único

# @markdown ### Caminhos
INPUT_PATH = "/content/drive/MyDrive/codename-rvc/inputs/audio.wav"  # @param {type:"string"}
OUTPUT_PATH = "/content/drive/MyDrive/codename-rvc/outputs/output.wav"  # @param {type:"string"}
MODEL_PATH = "logs/meu_modelo/meu_modelo.pth"  # @param {type:"string"}
INDEX_PATH = "logs/meu_modelo/added_meu_modelo.index"  # @param {type:"string"}

# @markdown ### Configurações
PITCH = 0  # @param {type:"slider", min:-24, max:24, step:1}
F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
INDEX_RATE = 0.5  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
PROTECT = 0.33  # @param {type:"slider", min:0.0, max:0.5, step:0.05}

SPLIT_AUDIO = False  # @param {type:"boolean"}
CLEAN_AUDIO = False  # @param {type:"boolean"}
F0_AUTOTUNE = False  # @param {type:"boolean"}
EXPORT_FORMAT = "WAV"  # @param ["WAV", "MP3", "FLAC", "OGG", "M4A"]

print(f"🎵 Input: {INPUT_PATH}")
print(f"💾 Output: {OUTPUT_PATH}")
print("="*60)

# Criar pasta de saída se não existir
from pathlib import Path
Path(OUTPUT_PATH).parent.mkdir(parents=True, exist_ok=True)

!python core.py infer \
    --input_path "{INPUT_PATH}" \
    --output_path "{OUTPUT_PATH}" \
    --pth_path "{MODEL_PATH}" \
    --index_path "{INDEX_PATH}" \
    --pitch {PITCH} \
    --f0_method "{F0_METHOD}" \
    --index_rate {INDEX_RATE} \
    --protect {PROTECT} \
    --split_audio {str(SPLIT_AUDIO)} \
    --clean_audio {str(CLEAN_AUDIO)} \
    --f0_autotune {str(F0_AUTOTUNE)} \
    --export_format "{EXPORT_FORMAT}"

print(f"\n✅ Salvo em: {OUTPUT_PATH}")

In [ ]:
# @title 9️⃣ Inferência em Lote

INPUT_FOLDER = "/content/drive/MyDrive/codename-rvc/inputs/"  # @param {type:"string"}
OUTPUT_FOLDER = "/content/drive/MyDrive/codename-rvc/outputs/"  # @param {type:"string"}
MODEL_PATH = "logs/meu_modelo/meu_modelo.pth"  # @param {type:"string"}
INDEX_PATH = "logs/meu_modelo/added_meu_modelo.index"  # @param {type:"string"}

PITCH = 0  # @param {type:"slider", min:-24, max:24, step:1}
F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
INDEX_RATE = 0.5  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
EXPORT_FORMAT = "WAV"  # @param ["WAV", "MP3", "FLAC", "OGG", "M4A"]

import os
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print(f"📂 Input: {INPUT_FOLDER}")
print(f"📂 Output: {OUTPUT_FOLDER}")
print("="*60)

!python core.py batch_infer \
    --input_folder "{INPUT_FOLDER}" \
    --output_folder "{OUTPUT_FOLDER}" \
    --pth_path "{MODEL_PATH}" \
    --index_path "{INDEX_PATH}" \
    --pitch {PITCH} \
    --f0_method "{F0_METHOD}" \
    --index_rate {INDEX_RATE} \
    --export_format "{EXPORT_FORMAT}"

print(f"\n✅ Arquivos convertidos em: {OUTPUT_FOLDER}")